<a href="https://colab.research.google.com/github/srikarraju/eGrocery_Demand_Prediction/blob/main/eGrocery_Env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment that simulates eGrocery store given the demand data

In [ ]:
import gym
from gym import spaces
import numpy as np

In [ ]:
class eGroceryEnv(gym.Env):
  def __init__(self, df, shelf_life, shortage_cost, wastage_cost, box_size, products_count=10, max_demand=100):
    self.df = df
    self.n_features = 3 + 4*products_count + len(shelf_life) + sum(shelf_life)
    self.products_count = products_count
    self.shelf_life = shelf_life
    self.shortage_cost = shortage_cost
    self.wastage_cost = wastage_cost
    self.box_size = box_size

    super(eGroceryEnv,self).__init__()

    self.action_spaces = spaces.MultiDiscrete([max_demand]*self.products_count)
    self.observation_space = spaces.Box(low = 0, high = 20000, shape=(self.n_features,), dtype = np.float16)

    self.ind_shortages = list([])
    self.ind_wastages = list([])
    self.all_shortages = list([])
    self.all_wastages = list([])

    self.rewards = list([])
    self.total_loss = list([])
    self.curr_step = 0

    self.curr_stock = list([])
    for i in range(self.products_count):
      self.curr_stock.append([])
      for j in range(self.shelf_life[i]):
        self.curr_stock[i].append(0)

  def step(self, action):

    # remove products out of shelf life from stock and update wastages
    wastages = []
    for i in range(self.products_count):
      wastages.append(self.curr_stock[i].pop(0))
      self.curr_stock[i].append(action[i]*self.box_size[i])
    #print(wastages)
    self.ind_wastages.append(wastages)
    self.all_wastages.append(sum(wastages))

    # Full fill the curr day demand and track shortages
    prods = ['prod'+str(i) for i in [8,11,15,17,94,95,96,110,112,128]]
    demands = self.df.loc[self.curr_step,prods]
    shortages = []
    for i in range(self.products_count):
      for j in range(self.shelf_life[i]):
        if self.curr_stock[i][j] >=demands[i]:
          self.curr_stock[i][j] -= demands[i]
          demands[i] = 0
          break
        else:
          demands[i] -= self.curr_stock[i][j]
          self.curr_stock[i][j] = 0
      #print(demands[i])
      shortages.append(int(demands[i]))
    self.ind_shortages.append(shortages)
    self.all_shortages.append(sum(shortages))


    # Cacluate overall reward
    self.curr_step += 1
    #print(wastages,shortages)
    reward = -1*(np.dot(self.wastage_cost,np.asarray(wastages))+np.dot(self.shortage_cost,np.asarray(shortages)))
    self.rewards.append(reward)

    # Update Next sate
    obs  = self.df.loc[self.curr_step]
    next_state = list([])
    for i in range(len(self.curr_stock)):
      for j in range(len(self.curr_stock[i])):
        next_state.append(self.curr_stock[i][j])
    obs = list(obs) + list(next_state) + list(self.shelf_life)

    done = (self.curr_step < 0) or (self.curr_step > self.df.shape[0]-2)


    return obs, done, reward, {}

  def reset(self):
		# Reset the state of the environment to an initial state
    self.ind_shortages = list([])
    self.ind_wastages = list([])
    self.all_shortages = list([])
    self.all_wasatges = list([])

    self.rewards = list([])
    self.total_loss = list([])
    self.curr_step = 0
    self.curr_stock = list([])
    for i in range(self.products_count):
      self.curr_stock.append([])
      for j in range(self.shelf_life[i]):
        self.curr_stock[i].append(0)

    return [0]*self.n_features


In [ ]:
env = eGroceryEnv(df, shelf_life=[2]*10, shortage_cost=np.asarray([1]*10), wastage_cost=np.asarray([1]*10), box_size=[2]*10, products_count=10, max_demand=100)

In [ ]:
import random
obs = env.reset()
for i in range(100):
  action = [random.randint(0,10) for j in range(10)]
  env.step(action)

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/eGrocery_Project

/content/drive/MyDrive/Colab Notebooks/eGrocery_Project


In [ ]:
from eGrocery_Env import eGroceryEnv